In [1]:
import pandas as pd
import numpy as np
import re

from datetime import date

import cpi #A Python library that quickly adjusts U.S. dollars for inflation using the Consumer Price Index (CPI).

## Load Disney 2016 Dataset

In [2]:
#Load Disney titles
disney_df = pd.read_csv('input/disney_movies_total_gross.csv')
disney_df.drop(columns='inflation_adjusted_gross', inplace=True)
disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000"


## Load Disney 2017 - 2022 Dataset

In [3]:
#Load the hand copied data from https://www.the-numbers.com/movies/distributor/Walt-Disney#tab=year

df_2021 = pd.read_csv('input/Disney_2017_to_2021.csv')
df_2021.drop(columns=['Rank','Tickets Sold'], inplace=True)
# df_2022['Release_Date']= pd.to_datetime(df_2022['Release_Date'])
# df_2022['year'] = pd.DatetimeIndex(df_2022['Release_Date']).year
df_2021.tail()

,Movie,Release_Date,Genre,MPAA_Rating,Gross
59,Raya and the Last Dragon,"Mar 5, 2021",Adventure,PG,"$54,723,032"
60,The Emperor's New Groove,"Dec 15, 2000",Adventure,G,"$334,000"
61,Remember the Titans,"Sep 29, 2000",Drama,PG,"$65,000"
62,Encanto,"Nov 24, 2021",Adventure,PG,"$4,620,741"
63,Eternals,"Nov 5, 2021",Action,PG-13,"$175,832"


In [4]:
df_2021.rename(columns={'Movie': 'movie_title', 'Release_Date': 'release_date', 'Genre': 'genre',\
                        'MPAA_Rating': 'MPAA_rating', 'Gross': 'total_gross'}, inplace=True)
df_2021.head()

,movie_title,release_date,genre,MPAA_rating,total_gross
0,Star Wars Ep. VIII: The Last Jedi,"Dec 15, 2017",Adventure,PG-13,"$517,218,368"
1,Beauty and the Beast,"Mar 17, 2017",Musical,PG,"$504,014,165"
2,Guardians of the Galaxy Vol 2,"May 5, 2017",Action,PG-13,"$389,813,101"
3,Thor: Ragnarok,"Nov 3, 2017",Action,PG-13,"$311,225,150"
4,Coco,"Nov 22, 2017",Musical,PG,"$179,834,199"


In [5]:
df_2021.count()

movie_title     64
release_date    64
genre           64
MPAA_rating     64
total_gross     64
dtype: int64

## Concat to get Disney dataset to 2021

In [6]:
#Concat df_2022 to Disney dataframe
all_disney_df = pd.concat([disney_df,df_2021])
all_disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000"


In [7]:
all_disney_df.tail(30)

,movie_title,release_date,genre,MPAA_rating,total_gross
34,Maleficent: Mistress of Evil,"Oct 18, 2019",Adventure,PG,"$113,294,737"
35,Mary Poppins Returns,"Dec 19, 2018",Musical,PG,"$33,141,176"
36,Ralph Breaks The Internet,"Nov 21, 2018",Adventure,PG,"$13,939,018"
37,Penguins,"Apr 17, 2019",Documentary,G,"$7,699,452"
38,The Nutcracker and the Four Realms,"Nov 2, 2018",Adventure,PG,"$73,093"
39,Star Wars: The Rise of Skywalker,"Dec 20, 2019",Adventure,PG-13,"$124,496,308"
40,Onward,"Mar 6, 2020",Adventure,PG,"$61,555,145"
41,Frozen II,"Nov 22, 2019",Adventure,PG,"$26,934,045"
42,Hocus Pocus,"Jul 16, 1993",Comedy,PG,"$4,777,471"
43,The Nightmare Before Christmas,"Oct 13, 1993",Musical,PG,"$2,286,000"


In [8]:
all_disney_df.count()

movie_title     643
release_date    643
genre           626
MPAA_rating     587
total_gross     643
dtype: int64

In [9]:
all_disney_df['total_gross'] = all_disney_df['total_gross'].str.replace(',','')
all_disney_df['total_gross'] = all_disney_df['total_gross'].str.replace('$','')
all_disney_df.head()

/Users/vuvie/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,movie_title,release_date,genre,MPAA_rating,total_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,184925485
1,Pinocchio,"Feb 9, 1940",Adventure,G,84300000
2,Fantasia,"Nov 13, 1940",Musical,G,83320000
3,Song of the South,"Nov 12, 1946",Adventure,G,65000000
4,Cinderella,"Feb 15, 1950",Drama,G,85000000


In [10]:
#create a year column for the movie list
all_disney_df['year'] = pd.DatetimeIndex(all_disney_df['release_date']).year
all_disney_df['year'] = all_disney_df['year'].astype(int)

all_disney_df['total_gross'] = all_disney_df['total_gross'].astype(int)
all_disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,year
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,184925485,1937
1,Pinocchio,"Feb 9, 1940",Adventure,G,84300000,1940
2,Fantasia,"Nov 13, 1940",Musical,G,83320000,1940
3,Song of the South,"Nov 12, 1946",Adventure,G,65000000,1946
4,Cinderella,"Feb 15, 1950",Drama,G,85000000,1950


In [11]:
# any duplicate movies? especially released in Nov / Dec of the year will collect gross the next year
all_disney_df.loc[all_disney_df.duplicated(subset=['movie_title','year','release_date'])]

,movie_title,release_date,genre,MPAA_rating,total_gross,year
8,Rogue One: A Star Wars Story,"Dec 16, 2016",Adventure,PG-13,107189617,2016
9,Moana,"Nov 23, 2016",Musical,PG,38710930,2016
11,Doctor Strange,"Nov 4, 2016",Action,PG-13,2534130,2016
12,Queen of Katwe,"Sep 23, 2016",Drama,PG,17078,2016
20,Star Wars Ep. VIII: The Last Jedi,"Dec 15, 2017",Adventure,PG-13,102963014,2017
24,Coco,"Nov 22, 2017",Musical,PG,29891816,2017
25,Thor: Ragnarok,"Nov 3, 2017",Action,PG-13,3833139,2017
35,Mary Poppins Returns,"Dec 19, 2018",Musical,PG,33141176,2018
36,Ralph Breaks The Internet,"Nov 21, 2018",Adventure,PG,13939018,2018
38,The Nutcracker and the Four Realms,"Nov 2, 2018",Adventure,PG,73093,2018


In [12]:
#check one title to see the duplicate
all_disney_df.loc[all_disney_df['movie_title'] =='Encanto']

,movie_title,release_date,genre,MPAA_rating,total_gross,year
57,Encanto,"Nov 24, 2021",Adventure,PG,91386126,2021
62,Encanto,"Nov 24, 2021",Adventure,PG,4620741,2021


In [13]:
#sum up the gross amount by same title & year. Example Encanto was released Year End 2021 
#therefore will continue to show in movie theatres throughout early 2022
sum_gross_disney_df = all_disney_df.groupby(['movie_title','year','release_date'],as_index=False)['total_gross'].sum()
sum_gross_disney_df.count()

movie_title     618
year            618
release_date    618
total_gross     618
dtype: int64

In [14]:
#make sure the total gross is added correctly
sum_gross_disney_df.loc[sum_gross_disney_df['movie_title'] =='Encanto']

,movie_title,year,release_date,total_gross
160,Encanto,2021,"Nov 24, 2021",96006867


In [15]:
# def inflate_column(data, column):
#     """
#     Adjust for inflation the series of values in column of the   
#     dataframe data
#     """
#     return data.apply(lambda x: cpi.inflate(x[column], 
#                       date(x.year, 1, 1), to=date(2022,1,1)), axis=1)

In [16]:
def inflate_column(data, column):
    """
    Adjust for inflation the series of values in column of the   
    dataframe data
    """
    return data.apply(lambda x: cpi.inflate(x[column], 
                      x.year, 2021), axis=1)

In [17]:
# ##cpi syntax
# cpi.inflate(107189617, date(2016, 1, 1), to=date(2022,1,1))
# cpi.inflate(107189617, 2016, 2021)

In [18]:
# adjust the Dosmestic Box Office total_gross to today's inflation numbers 
sum_gross_disney_df['inflation_adjusted_gross'] = inflate_column(sum_gross_disney_df, 'total_gross')
sum_gross_disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1.386569e+09
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,2.352021e+08
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,1.053377e+08
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,1.371176e+07
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,2.840652e+08


In [19]:
sum_gross_disney_df.loc[sum_gross_disney_df['movie_title'] =='Pinocchio']

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
351,Pinocchio,1940,"Feb 9, 1940",84300000,1.631627e+09


In [20]:
# change exponential numbers to natural numbers

sum_gross_disney_df['inflation_adjusted_gross'] = sum_gross_disney_df['inflation_adjusted_gross'].astype('int64')
sum_gross_disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


## Movie name changes to match IMDB as closely as possible

In [21]:
sum_gross_disney_df['movie_title'] = sum_gross_disney_df['movie_title'].str.replace('Ep.','Episode')
sum_gross_disney_df.head()

/Users/vuvie/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [22]:
sum_gross_disney_df[sum_gross_disney_df.movie_title.str.startswith('Who Framed')]

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
607,Who Framed Roger Rabbit?,1988,"Jun 22, 1988",154112492,352999678


In [27]:
sum_gross_disney_df[sum_gross_disney_df.movie_title.str.startswith('Star')]

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
422,Star Wars Episode V: The Empire Strikes Back,2020,"Sep 25, 2020",1580209,1654447
423,Star Wars Episode VII: The Force Awakens,2015,"Dec 18, 2015",936662225,1070840332
424,Star Wars Episode VIII: The Last Jedi,2017,"Dec 15, 2017",620181382,685584811
425,Star Wars: The Rise of Skywalker,2019,"Dec 20, 2019",515202542,546061452


## Export complete list

In [24]:
# Export complete list
sum_gross_disney_df.to_csv (r'output/2021_disney_movies_total_gross.csv', index = None, header=True) 

## Extra codes below. Not required

In [25]:
# value=str('1.631627e+09').replace(',', '.')
# float(value)

In [26]:
# for i in sum_gross_disney_df.index:
#     print(i)
#     inflation=str(sum_gross_disney_df['inflation_gross'][i]).replace(',', '.')
#     print(inflation)
#     sum_gross_disney_df['inflation_adjusted_gross'][i] = float(inflation)